# Cronologia degli Esperimenti

In [18]:
import plotly.express as px
import pandas as pd

# Lettura csv degli esperimenti
df = pd.read_csv('./experiments_collector.csv', parse_dates=["Start Date", "End Date"])
# df.dtypes
# Esperimenti solo con corrente positiva
df = df.loc[df['Min Current [A/cm2]'] >= 0]

# Range di corrente
df['e_j_plus'] = df['Max Current [A/cm2]'] - df['Avg Current [A/cm2]']
df['e_j_minus'] = df['Avg Current [A/cm2]'] - df['Min Current [A/cm2]']
df['range_j'] = df['Max Current [A/cm2]'] - df['Min Current [A/cm2]']

# Range di temperatura
df['e_t_plus'] = df['Max Temperature [K]'] - df['Avg Temperature [K]']
df['e_t_minus'] = df['Avg Temperature [K]'] - df['Min Temperature [K]']
df['range_t'] = df['Max Temperature [K]'] - df['Min Temperature [K]']

In [19]:
df['Label'] = df['Oscillation val']/100
#df['Avg Current [uA/cm2]'] = df['Avg Current [A/cm2]']*1e6
df[['Avg Temperature [K]', 'Avg Current [A/cm2]', 'Label']]

,Avg Temperature [K],Avg Current [A/cm2],Label
0,94.685079,0.000179,0.0
1,91.220244,0.000018,0.0
2,294.020829,2.948843,0.0
3,166.925348,0.019550,0.0
4,89.233235,0.000011,0.0
...,...,...,...
131,116.802982,0.000059,0.0
132,115.784791,0.000059,0.0
133,115.314016,0.000059,0.0
134,116.229975,0.000059,0.0


In [40]:
import plotly.graph_objects as go
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Load and split data
X = df[['Avg Temperature [K]', 'Avg Current [A/cm2]']].values
y = df[['Label']].values #.transpose()
y = y.reshape(-1)
#print(X.shape, y.shape)

# Load and split data
#X, y = make_moons(noise=0.3, random_state=0)
#print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y.astype(str), test_size=0.25, random_state=0)

# Fit the model on training data, predict on test data
clf = KNeighborsClassifier(15)
clf.fit(X_train, y_train)
y_score = clf.predict_proba(X_test)[:, 1]

# print(X_test)

fig = px.scatter(
    X_test, x=0, y=1,
    color=y_score, 
    #color_continuous_scale='RdBu', 
    log_y=True, 
    symbol=y_test, 
    #symbol_map={'0': 'square-dot', '1': 'circle-dot'},
    labels={'symbol': 'label', 'color': 'score of <br>first class', 
            '0' :'Avg Temperature [K]', '1':'Avg Current [A/cm2]'}
)
fig.update_traces(marker_size=12, marker_line_width=1.5)
fig.update_layout(legend_orientation='h')
fig.show()
